In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer,TFBertForSequenceClassification
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [55]:
df = pd.read_csv("C:\\Users\\thebl\\Documents\\Lomba\\SatDat2024\\FullSenyum-SatriaData2024\\DataSet\\cleaned_final_label.csv")
df

,Unnamed: 0,text,label,stemmed_text,label_angka
0,0,kunjungan prabowo untuk meresmikan menyerahkan...,Sumber Daya Alam,kunjung prabowo untuk resmi serah proyek bantu...,5
1,1,anies tepuk tangan meriah jadi rektor mewajibk...,Politik,anies tepuk tangan riah jadi rektor wajib mata...,0
2,2,emng bener sih pendukung anies juga pendukung ...,Demografi,emng bener sih dukung anies juga dukung prabow...,6
3,3,sewaktu anies bersikap kritis kinerja pak prab...,Politik,waktu anies sikap kritis kerja pak prabowo ang...,0
4,4,anies baswedan harap asn termasuk tni polri pe...,Politik,anies baswedan harap asn masuk tni polri pegan...,0
...,...,...,...,...,...
4578,4578,ngeliat debat kemaren pas prabowo kicep kekira...,Politik,ngeliat debat kemaren pas prabowo kicep kira k...,0
4579,4579,masyarakat yakin prabowo gibran memiliki visi ...,Politik,masyarakat yakin prabowo gibran milik visi jal...,0
4580,4580,imo both are irrational but satu jauh lebih ir...,Ekonomi,imo both are irrational but satu jauh lebih ir...,4
4581,4581,look at that pak ganjar sudah berkecimpung lgi...,Pertahanan dan Keamanan,look at that pak ganjar sudah kecimpung lgisla...,3


In [56]:
df['stemmed_text'].duplicated().sum()

312

In [57]:
df[df['stemmed_text'].duplicated()]

,Unnamed: 0,text,label,stemmed_text,label_angka
59,59,sewaktu anies bersikap kritis kinerja pak prab...,Politik,waktu anies sikap kritis kerja pak prabowo ang...,0
126,126,bulan januari serangan kubu dkk semakin sengit...,Pertahanan dan Keamanan,bulan januari serang kubu dkk makin sengit mas...,3
141,141,bulan januari serangan kubu dkk semakin sengit...,Pertahanan dan Keamanan,bulan januari serang kubu dkk makin sengit mas...,3
147,147,bulan januari serangan kubu dkk semakin sengit...,Pertahanan dan Keamanan,bulan januari serang kubu dkk makin sengit mas...,3
148,148,capres cawapres ganjar pranowo mahfud md menga...,Ideologi,capres cawapres ganjar pranowo mahfud md ajak ...,2
...,...,...,...,...,...
4520,4520,netizen geger haikal hassan gunakan politik id...,Politik,netizen geger haikal hassan guna politik ident...,0
4522,4522,cukup lihat track recordnya ganjar pranowo mah...,Politik,cukup lihat track recordnya ganjar pranowo mah...,0
4544,4544,bukan cuma tampilan luarnya kebijakan ganjar p...,Ideologi,bukan cuma tampil luar bijak ganjar pranowo ma...,2
4574,4574,bulan januari serangan kubu dkk semakin sengit...,Pertahanan dan Keamanan,bulan januari serang kubu dkk makin sengit mas...,3


In [58]:
df = df.drop(columns=['Unnamed: 0'])

In [59]:
df.duplicated().sum()

286

In [60]:
df= df.drop_duplicates(subset='stemmed_text')

In [61]:
df.duplicated().sum()

0

In [62]:
df.shape

(4271, 4)

In [63]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize text
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='tf')

# Apply tokenization
tokenized_datasets = df['stemmed_text'].apply(tokenize_function).tolist()

In [64]:
input_ids = np.array([x['input_ids'].numpy() for x in tokenized_datasets])
attention_mask = np.array([x['attention_mask'].numpy() for x in tokenized_datasets])
labels = df['label_angka'].values

In [65]:
smote = SMOTE(random_state=42)
input_ids_resampled, labels_resampled = smote.fit_resample(input_ids.reshape(-1, input_ids.shape[-1]), labels)
attention_mask_resampled, _ = smote.fit_resample(attention_mask.reshape(-1, attention_mask.shape[-1]), labels)

input_ids_resampled = input_ids_resampled.reshape(-1, input_ids.shape[-1])
attention_mask_resampled = attention_mask_resampled.reshape(-1, attention_mask.shape[-1])

# Convert labels to categorical format
labels_resampled = to_categorical(labels_resampled, num_classes=8)


In [66]:
X_train, X_val, y_train, y_val = train_test_split(input_ids_resampled, labels_resampled, test_size=0.2, random_state=42)


In [67]:
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_train, 'attention_mask': attention_mask_train}, y_train))
train_dataset = train_dataset.batch(16)

eval_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_val, 'attention_mask': attention_mask_val}, y_val))
eval_dataset = eval_dataset.batch(16)


NameError: name 'attention_mask_train' is not defined

In [ ]:
from transformers import TFBertForSequenceClassification

# Initialize BERT model
bert_model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=8)

# Compile the model with optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.CategoricalAccuracy('accuracy')]

bert_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
history = bert_model.fit(train_dataset, epochs=3, validation_data=eval_dataset)
eval_results = bert_model.evaluate(eval_dataset)
print(f'Evaluation results: {eval_results}')

In [70]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize text
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='tf')

# Apply tokenization
tokenized_datasets = df['stemmed_text'].apply(tokenize_function).tolist()

# Extract input_ids, attention_mask, and labels
input_ids = np.array([x['input_ids'].numpy()[0] for x in tokenized_datasets])
attention_mask = np.array([x['attention_mask'].numpy()[0] for x in tokenized_datasets])
labels = df['label_angka'].values

# Apply SMOTE on tokenized inputs
smote = SMOTE(random_state=42)
input_ids_resampled, labels_resampled = smote.fit_resample(input_ids, labels)
attention_mask_resampled, _ = smote.fit_resample(attention_mask, labels)

# Convert labels to categorical format
labels_resampled = to_categorical(labels_resampled, num_classes=8)

# Split the resampled data into training and evaluation sets
X_train, X_val, y_train, y_val = train_test_split(input_ids_resampled, labels_resampled, test_size=0.2, random_state=42)
attention_mask_train, attention_mask_val = train_test_split(attention_mask_resampled, test_size=0.2, random_state=42)

# Convert data to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_train, 'attention_mask': attention_mask_train}, y_train))
train_dataset = train_dataset.batch(16)

eval_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_val, 'attention_mask': attention_mask_val}, y_val))
eval_dataset = eval_dataset.batch(16)

# Initialize model
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=8)

# Compile the model with optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.CategoricalAccuracy('accuracy')]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
history = model.fit(train_dataset, epochs=3, validation_data=eval_dataset)

# Evaluate the model
eval_results = model.evaluate(eval_dataset)
print(f'Evaluation results: {eval_results}')

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'Variable' object has no attribute '_distribute_strategy'

In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, TFBertForSequenceClassification
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Load your dataset
# df = pd.read_csv('your_dataset.csv')
# Ensure df has 'stemmed_text' and 'label_angka' columns

# Initialize tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize text
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='np')

# Apply tokenization
tokenized_datasets = df['stemmed_text'].apply(tokenize_function).tolist()

# Extract input_ids, attention_mask, and labels
input_ids = np.array([x['input_ids'][0] for x in tokenized_datasets])
attention_mask = np.array([x['attention_mask'][0] for x in tokenized_datasets])
labels = df['label_angka'].values

# Apply SMOTE on tokenized inputs
smote = SMOTE(random_state=42)
input_ids_resampled, labels_resampled = smote.fit_resample(input_ids, labels)
attention_mask_resampled, _ = smote.fit_resample(attention_mask, labels)

# Convert labels to categorical format
labels_resampled = to_categorical(labels_resampled, num_classes=8)

# Split the resampled data into training and evaluation sets
X_train, X_val, y_train, y_val = train_test_split(input_ids_resampled, labels_resampled, test_size=0.2, random_state=42)
attention_mask_train, attention_mask_val = train_test_split(attention_mask_resampled, test_size=0.2, random_state=42)

# Convert data to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_train, 'attention_mask': attention_mask_train}, y_train))
train_dataset = train_dataset.batch(16)

eval_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_val, 'attention_mask': attention_mask_val}, y_val))
eval_dataset = eval_dataset.batch(16)

# Initialize model
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=8)

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Training loop using tf.GradientTape
@tf.function
def train_step(input_ids, attention_mask, labels):
    with tf.GradientTape() as tape:
        logits = model(input_ids, attention_mask=attention_mask, training=True)[0]
        loss = loss_fn(labels, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training
for epoch in range(3):
    print(f'Epoch {epoch+1}/{3}')
    for batch in train_dataset:
        input_ids = batch[0]['input_ids']
        attention_mask = batch[0]['attention_mask']
        labels = batch[1]
        loss = train_step(input_ids, attention_mask, labels)
        print(f'Loss: {loss.numpy()}')

# Evaluation
def evaluate(dataset):
    total_loss = 0
    total_accuracy = 0
    num_batches = 0
    for batch in dataset:
        input_ids = batch[0]['input_ids']
        attention_mask = batch[0]['attention_mask']
        labels = batch[1]
        logits = model(input_ids, attention_mask=attention_mask, training=False)[0]
        loss = loss_fn(labels, logits)
        total_loss += loss.numpy()
        predictions = tf.argmax(logits, axis=-1)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, tf.argmax(labels, axis=-1)), tf.float32))
        total_accuracy += accuracy.numpy()
        num_batches += 1
    return total_loss / num_batches, total_accuracy / num_batches

eval_loss, eval_accuracy = evaluate(eval_dataset)
print(f'Evaluation Loss: {eval_loss}')
print(f'Evaluation Accuracy: {eval_accuracy}')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3

Loss: 2.124677896499634
Loss: 2.23899507522583
Loss: 2.064851760864258
Loss: 2.0310683250427246
Loss: 2.0390005111694336
Loss: 2.0471906661987305
Loss: 2.0214178562164307
Loss: 2.1140971183776855
Loss: 2.1171064376831055
Loss: 2.0170905590057373
Loss: 2.038987636566162
Loss: 2.1253232955932617
Loss: 2.038936138153076
Loss: 2.0582666397094727
Loss: 2.0780587196350098
Loss: 2.038452386856079
Loss: 2.0860068798065186
Loss: 1.9561851024627686
Loss: 2.0551905632019043
Loss: 1.9892600774765015
Loss: 2.0119781494140625
Loss: 2.0600168704986572
Loss: 2.0379233360290527
Loss: 2.0168566703796387
Loss: 2.0112318992614746
Loss: 1.9914896488189697
Loss: 2.023820400238037
Loss: 2.0916481018066406
Loss: 2.0121521949768066
Loss: 2.020667552947998
Loss: 1.9617750644683838
Loss: 2.0965704917907715
Loss: 1.9668402671813965
Loss: 2.004055976867676
Loss: 1.9880656003952026
Loss: 1.88955557346344
Loss: 1.9483811855316162
Loss: 2.010159492492676
Loss: 2.0598039627075195
Loss: 1.6285480260849
Loss:

KeyboardInterrupt: 

In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, TFBertForSequenceClassification
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Load your dataset
# df = pd.read_csv('your_dataset.csv')
# Ensure df has 'stemmed_text' and 'label_angka' columns

# Initialize tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Perform oversampling on the raw text data
ros = RandomOverSampler(random_state=42)
texts_resampled, labels_resampled = ros.fit_resample(df['stemmed_text'].values.reshape(-1, 1), df['label_angka'].values)
texts_resampled = texts_resampled.flatten()

# Tokenize text
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='np')

# Apply tokenization
tokenized_datasets = [tokenize_function(text) for text in texts_resampled]

# Extract input_ids, attention_mask, and labels
input_ids = np.array([x['input_ids'][0] for x in tokenized_datasets])
attention_mask = np.array([x['attention_mask'][0] for x in tokenized_datasets])
labels_resampled = to_categorical(labels_resampled, num_classes=8)

# Split the resampled data into training and evaluation sets
X_train, X_val, y_train, y_val = train_test_split(input_ids, labels_resampled, test_size=0.2, random_state=42)
attention_mask_train, attention_mask_val = train_test_split(attention_mask, test_size=0.2, random_state=42)

# Convert data to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_train, 'attention_mask': attention_mask_train}, y_train))
train_dataset = train_dataset.batch(16)

eval_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_val, 'attention_mask': attention_mask_val}, y_val))
eval_dataset = eval_dataset.batch(16)

# Initialize model
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=8)

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Training loop using tf.GradientTape
@tf.function
def train_step(input_ids, attention_mask, labels):
    with tf.GradientTape() as tape:
        logits = model(input_ids, attention_mask=attention_mask, training=True)[0]
        loss = loss_fn(labels, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training
for epoch in range(3):
    print(f'Epoch {epoch+1}/{3}')
    for batch in train_dataset:
        input_ids = batch[0]['input_ids']
        attention_mask = batch[0]['attention_mask']
        labels = batch[1]
        loss = train_step(input_ids, attention_mask, labels)
        print(f'Loss: {loss.numpy()}')

# Evaluation
def evaluate(dataset):
    total_loss = 0
    total_accuracy = 0
    num_batches = 0
    for batch in dataset:
        input_ids = batch[0]['input_ids']
        attention_mask = batch[0]['attention_mask']
        labels = batch[1]
        logits = model(input_ids, attention_mask=attention_mask, training=False)[0]
        loss = loss_fn(labels, logits)
        total_loss += loss.numpy()
        predictions = tf.argmax(logits, axis=-1)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, tf.argmax(labels, axis=-1)), tf.float32))
        total_accuracy += accuracy.numpy()
        num_batches += 1
    return total_loss / num_batches, total_accuracy / num_batches

eval_loss, eval_accuracy = evaluate(eval_dataset)
print(f'Evaluation Loss: {eval_loss}')
print(f'Evaluation Accuracy: {eval_accuracy}')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
Loss: 2.129279136657715
Loss: 2.116427421569824
Loss: 2.1419947147369385
Loss: 2.0599160194396973
Loss: 2.217255115509033
Loss: 2.0580785274505615
Loss: 2.116018772125244
Loss: 2.0919649600982666
Loss: 2.0816457271575928
Loss: 2.1161999702453613
Loss: 2.081799268722534
Loss: 2.1426610946655273
Loss: 2.08425235748291
Loss: 2.0863564014434814
Loss: 2.1414875984191895
Loss: 2.1100540161132812
Loss: 2.1504645347595215
Loss: 2.032630681991577
Loss: 2.101759672164917
Loss: 2.0692126750946045
Loss: 2.1306400299072266
Loss: 2.1066277027130127
Loss: 2.185425281524658
Loss: 2.1067886352539062
Loss: 2.0948374271392822
Loss: 2.0500545501708984
Loss: 2.0723071098327637
Loss: 2.1084060668945312
Loss: 2.0574021339416504
Loss: 2.151772975921631
Loss: 2.0436043739318848
Loss: 2.139460563659668
Loss: 2.0520994663238525
Loss: 2.075068950653076
Loss: 2.081045627593994
Loss: 2.0998120307922363
Loss: 2.0629658699035645
Loss: 2.130089282989502
Loss: 2.1184377670288086
Loss: 2.099052906036377
Loss: 

KeyboardInterrupt: 

In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Load your dataset
# df = pd.read_csv('your_dataset.csv')
# Ensure df has 'stemmed_text' and 'label_angka' columns

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
X_tfidf = tfidf_vectorizer.fit_transform(df['stemmed_text'])

# Apply SMOTE on TF-IDF features
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, df['label_angka'])

# Tokenize text using BERT tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='np')

# Assuming you convert X_resampled back to text here (not shown)

# Tokenize text
tokenized_datasets = [tokenize_function(text) for text in texts_resampled]

# Extract input_ids, attention_mask, and labels
input_ids = np.array([x['input_ids'][0] for x in tokenized_datasets])
attention_mask = np.array([x['attention_mask'][0] for x in tokenized_datasets])
labels_resampled = to_categorical(y_resampled, num_classes=8)

# Split the resampled data into training and evaluation sets
X_train, X_val, y_train, y_val = train_test_split(input_ids, labels_resampled, test_size=0.2, random_state=42)
attention_mask_train, attention_mask_val = train_test_split(attention_mask, test_size=0.2, random_state=42)

# Convert data to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_train, 'attention_mask': attention_mask_train}, y_train))
train_dataset = train_dataset.batch(16)

eval_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_val, 'attention_mask': attention_mask_val}, y_val))
eval_dataset = eval_dataset.batch(16)

# Initialize model
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=8)

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Training loop using tf.GradientTape
@tf.function
def train_step(input_ids, attention_mask, labels):
    with tf.GradientTape() as tape:
        logits = model(input_ids, attention_mask=attention_mask, training=True)[0]
        loss = loss_fn(labels, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training
for epoch in range(3):
    print(f'Epoch {epoch+1}/{3}')
    for batch in train_dataset:
        input_ids = batch[0]['input_ids']
        attention_mask = batch[0]['attention_mask']
        labels = batch[1]
        loss = train_step(input_ids, attention_mask, labels)
        print(f'Loss: {loss.numpy()}')

# Evaluation
def evaluate(dataset):
    total_loss = 0
    total_accuracy = 0
    num_batches = 0
    for batch in dataset:
        input_ids = batch[0]['input_ids']
        attention_mask = batch[0]['attention_mask']
        labels = batch[1]
        logits = model(input_ids, attention_mask=attention_mask, training=False)[0]
        loss = loss_fn(labels, logits)
        total_loss += loss.numpy()
        predictions = tf.argmax(logits, axis=-1)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, tf.argmax(labels, axis=-1)), tf.float32))
        total_accuracy += accuracy.numpy()
        num_batches += 1
    return total_loss / num_batches, total_accuracy / num_batches

eval_loss, eval_accuracy = evaluate(eval_dataset)
print(f'Evaluation Loss: {eval_loss}')
print(f'Evaluation Accuracy: {eval_accuracy}')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
Loss: 2.0731682777404785
Loss: 2.082730293273926
Loss: 2.145451068878174
Loss: 2.0821146965026855
Loss: 2.081237316131592
Loss: 2.0630857944488525
Loss: 2.0679216384887695
Loss: 2.137988328933716
Loss: 2.054811954498291
Loss: 2.0833520889282227
Loss: 2.107038974761963
Loss: 2.1097846031188965
Loss: 2.0546927452087402
Loss: 2.152414560317993
Loss: 2.176971435546875
Loss: 2.106440782546997
Loss: 2.1655662059783936
Loss: 2.0967016220092773
Loss: 2.0924875736236572
Loss: 2.057528495788574
Loss: 2.098031997680664
Loss: 2.1188442707061768
Loss: 2.1649889945983887
Loss: 2.094649314880371
Loss: 2.0693411827087402
Loss: 2.129472255706787
Loss: 2.1079261302948
Loss: 2.098616123199463
Loss: 2.0925140380859375
Loss: 2.102987766265869
Loss: 2.077183723449707
Loss: 2.1027705669403076
Loss: 2.1246719360351562
Loss: 2.0924949645996094
Loss: 2.0994088649749756
Loss: 2.1103172302246094
Loss: 2.1360013484954834
Loss: 2.12505841255188
Loss: 2.1091299057006836
Loss: 2.0911402702331543
Loss: 2.050

KeyboardInterrupt: 